In [ ]:
# only Sheet that is  Updating for Nepali Cars sheet

# import pandas as pd

# # df = pd.read_csv('output_data_final.csv');
# sheet_name_or_index = 1
# df = pd.read_excel('output_data_final.xlsx',sheet_name='Nepali Cars Data')
# brandName = []
# for index, eachRow in df.iterrows():
# # for title in df.title.head():
#     title=eachRow['title']
#     brandName=(title.split()[1])
#     year=(title.split()[0])
#     model = ' '.join(title.split()[2:4]) if len(title.split()) > 2 else ''
#     df.at[index, 'brandName'] = brandName
#     df.at[index, 'Make Year'] = year
#     df.at[index, 'Model'] = model
#     df.to_excel('update_nepali_cars_column.xlsx', index=False)

In [8]:
import pandas as pd
from xlsxwriter import Workbook

dfs = pd.read_excel('output_data_final.xlsx', sheet_name=None)

# Specify the sheet you want to update
updating_sheet = 'Nepali Cars Data'

# Extract the DataFrame for the specified sheet
df = dfs[updating_sheet]
brandName = []
for index, eachRow in df.iterrows():
    title = eachRow['title']
    brandName = title.split()[1]
    year = title.split()[0]
    model = ' '.join(title.split()[2:4]) if len(title.split()) > 2 else ''
    df.at[index, 'brandName'] = brandName
    df.at[index, 'Make Year'] = year
    df.at[index, 'Model'] = model

# Save the updated DataFrame back to the Excel file
with pd.ExcelWriter('v1-output-data-updated-column-nepalicars.xlsx', engine='xlsxwriter') as writer:
    for sheet_name, df_sheet in dfs.items():
        if sheet_name == updating_sheet:
            # Save the updated DataFrame for the specified sheet
            df.to_excel(writer, sheet_name=sheet_name, index=False)
        else:
            # Save the unchanged DataFrames for other sheets
            df_sheet.to_excel(writer, sheet_name=sheet_name, index=False)

In [14]:
brandName

['Mitsubishi', 'Hyundai', 'Suzuki', 'Suzuki', 'Renault']